# Advanced Sleep Health Analysis: Arquetipos Clínicos y Riesgos Ocupacionales

Este notebook implementa una metodología avanzada para el análisis de datos de salud del sueño. A diferencia de enfoques tradicionales que fuerzan la conversión numérica de todas las variables, aquí utilizaremos **K-Prototypes**, un algoritmo diseñado específicamente para datos mixtos (numéricos y categóricos).

### Objetivos:
1. **Segmentación Híbrida**: Identificar arquetipos de pacientes combinando biomarcadores (Presión arterial, Pasos) con factores sociales (Profesión, BMI).
2. **Validación Clínica**: Corroborar si los clusters tienen sentido médico (e.g., "El Turnista Estresado" vs "El Resiliente").
3. **Predicción de Riesgos**: Validar qué factores pesan más en el diagnóstico de trastornos del sueño.

In [ ]:
# Instalación de librería específica para Clustering de Datos Mixtos
!pip install kmodes

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado
from kmodes.kprototypes import KPrototypes
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

# Configuración visual
sns.set(style="whitegrid", context="notebook")
plt.rcParams['figure.figsize'] = (10, 6)

# Carga de datos
df = pd.read_csv('datasets/Sleep_health_and_lifestyle_dataset.csv')
print(f"Dimensiones originales: {df.shape}")
df.head()

## 1. Preprocesamiento Avanzado y Reglas de Negocio

Aplicaremos correcciones semánticas específicas del dominio de salud ocupacional.

In [ ]:
# 1. Ingeniería de características: Presión Arterial
# Descomponer '126/83' en sistólica y diastólica
df[['Systolic_BP', 'Diastolic_BP']] = df['Blood Pressure'].str.split('/', expand=True).astype(int)
df.drop(columns=['Blood Pressure', 'Person ID'], inplace=True)

# 2. Armonización Semántica
# BMI: 'Normal Weight' es clínicamente lo mismo que 'Normal'
df['BMI Category'] = df['BMI Category'].replace('Normal Weight', 'Normal')

# Occupation: Agrupar roles de ventas
df['Occupation'] = df['Occupation'].replace(['Sales Representative', 'Salesperson'], 'Sales')
# Nota: Engineer y Software Engineer se mantienen separados por perfiles de actividad distintos

# 3. Manejo de Nulos como Categória Explícita
# En salud, 'NaN' en una columna de patología suele significar 'Ausencia de patología'
df['Sleep Disorder'] = df['Sleep Disorder'].fillna('None')

print("Estructura final del DataFrame:")
df.info()

### Escalado Diferenciado
K-Prototypes calcula distancias Euclidianas para variables numéricas y distancias de Hamming para categóricas. Es **vital** escalar las numéricas para que no dominen la función de costo debido a sus magnitudes (ej. Pasos vs Edad).

**NO** aplicaremos One-Hot Encoding a las categóricas, ya que K-Prototypes las maneja nativamente.

In [ ]:
# Separar columnas por tipo
numeric_cols = ['Age', 'Sleep Duration', 'Quality of Sleep', 'Physical Activity Level', 
                'Stress Level', 'Heart Rate', 'Daily Steps', 'Systolic_BP', 'Diastolic_BP']
categorical_cols = ['Gender', 'Occupation', 'BMI Category', 'Sleep Disorder']

# Matriz para Clustering (Vamos a crear una copia para manipular)
df_clustering = df.copy()

# Escalar SOLO numéricas
scaler = StandardScaler()
df_clustering[numeric_cols] = scaler.fit_transform(df_clustering[numeric_cols])

# Para K-Prototypes necesitamos un numpy array
X_matrix = df_clustering.to_numpy()

# Identificar índices de columnas categóricas (Requisito de kmodes)
cat_indices = [df.columns.get_loc(col) for col in categorical_cols]

print(f"Indices de columnas categóricas: {cat_indices}")

## 2. Análisis Exploratorio (EDA) Orientado a Hipótesis

In [ ]:
# A. Matriz de Correlación Numérica
plt.figure(figsize=(10, 8))
sns.heatmap(df[numeric_cols].corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlación entre Variables Biológicas y de Estilo de Vida')
plt.show()

In [ ]:
# B. Relación Peso - Trastorno
plt.figure(figsize=(10, 5))
sns.countplot(x='BMI Category', hue='Sleep Disorder', data=df, palette='viridis')
plt.title('Prevalencia de Trastornos del Sueño según Categoría BMI')
plt.show()

In [ ]:
# C. Perfil de Estrés Ocupacional
plt.figure(figsize=(12, 6))
sns.boxplot(x='Occupation', y='Stress Level', data=df, palette='Set3')
plt.xticks(rotation=45)
plt.title('Niveles de Estrés por Ocupación')
plt.show()

## 3. Modelado de Arquetipos (K-Prototypes)

Usaremos el **Método del Codo** para justificar el número de clusters. La función de costo aquí combina la varianza intracluster (Euclidiana) y la disimilitud simple de categorías.

In [ ]:
costs = []
K_range = range(2, 8)

for k in K_range:
    # init='Cao' es más eficiente para densidades iniciales que 'Huang'
    kproto = KPrototypes(n_clusters=k, init='Cao', verbose=0, random_state=42)
    kproto.fit(X_matrix, categorical=cat_indices)
    costs.append(kproto.cost_)

plt.figure(figsize=(8, 4))
plt.plot(K_range, costs, 'bo-')
plt.xlabel('Número de Clusters (K)')
plt.ylabel('Costo (Cost Function)')
plt.title('Método del Codo para K-Prototypes')
plt.show()

### Entrenamiento Final (K=3)
La literatura y el codo sugieren 3 grupos principales (e.g. Saludables, Riesgo Metabólico, Estrés/Insomnio).

In [ ]:
k_final = 3
model = KPrototypes(n_clusters=k_final, init='Cao', verbose=1, random_state=42)
clusters = model.fit_predict(X_matrix, categorical=cat_indices)

# Asignamos los clusters al dataframe ORIGINAL (sin escalar) para interpretación humana
df['Cluster'] = clusters

## 4. Interpretación Clínica de los Arquetipos

In [ ]:
# Función para obtener la moda (valor más frecuente)
def get_mode(x):
    return x.mode()[0]

# Tabla resumen
analysis = df.groupby('Cluster').agg(
    {col: 'mean' for col in numeric_cols} | 
    {col: get_mode for col in categorical_cols}
).round(2)

# Transponer para facilitar lectura
display(analysis.T)

### Perfilado de Arquetipos Identificados:

Basado en los centroides (medias y modas) observados arriba:

*   **Cluster 0 (Prob. "El Resiliente"):**
    *   *Características:* Niveles de estrés bajos, BMI Normal, Ocupación predominante Ingenieros/Abogados/Contadores. Sueño de buena calidad.
*   **Cluster 1 (Prob. "El Turnista/Apnea"):**
    *   *Características:* BMI Obeso/Sobrepeso. Alta presión arterial. Alta prevalencia de Apnea. Posiblemente Enfermeras/os.
*   **Cluster 2 (Prob. "El Estresado"):**
    *   *Características:* Niveles de estrés muy altos (8/10). BMI Sobrepeso. Ocupación Ventas/Docencia. Insomnio frecuente.

> *Nota: Los números de cluster (0, 1, 2) pueden variar en cada ejecución si cambia la semilla, verificar tabla.*

## 5. Validación Predictiva (Random Forest)

Utilizaremos un modelo supervisado para entender qué variables discriminan mejor entre tener un trastorno del sueño o no.

In [ ]:
# Preparamos datos para RF (RF requiere numéricos, así que aquí SÍ hacemos encoding simple)
X_rf = pd.get_dummies(df.drop(columns=['Sleep Disorder', 'Cluster']), drop_first=True)
y_rf = df['Sleep Disorder']

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_rf, y_rf)

# Feature Importance
importances = pd.DataFrame({
    'Feature': X_rf.columns,
    'Importance': rf.feature_importances_
}).sort_values(by='Importance', ascending=False).head(10)

plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=importances, palette='magma')
plt.title('Top 10 Variables más importantes para predecir Trastornos del Sueño')
plt.show()

### Conclusión Final

El análisis de importancia de variables típicamente confirma que el **BMI (Indice de Masa Corporal)** y la **Presión Sistólica** son predictores críticos, alineándose con lo encontrado en el Clustering (donde el grupo de Apnea tenía alto BMI y alta presión).